In [2]:
# import modules
import librosa
import sklearn as skl
import numpy as np
import scipy as sp

In [5]:
# load audio data, audio data 'Y2' has 1018 examples, each of which is a 1s interval of swarming 
# audio data 'Y0' has 1018 examples, each of which is a 1s interval of not swarming.
matrix = sp.io.loadmat('Y2.mat')
data = matrix['Y']
print("Swarming data shape:",data.shape)
matrix1 = sp.io.loadmat('Y0.mat')
neg_data = matrix1['Y']
print("Non-swarming data shape:",neg_data.shape)

Swarming data shape: (44100, 1018)
Non-swarming data shape: (44100, 1018)


In [6]:
# librosa example
features = librosa.feature.mfcc(y=data[:,1],sr=44100,n_mfcc=12,n_fft=880,n_mels = 64,hop_length = 550)
features = features.flatten()
print("Feature shape",features.shape)

Feature shape (972,)


In [7]:
# extract features using librosa's MFCC
# for positive examples
feature_arr = np.zeros((np.size(features),data.shape[1]))
for i in range(data.shape[1]):
    feature = librosa.feature.mfcc(y=data[:,i],sr=44100,n_mfcc=12,n_fft=880,n_mels = 64,hop_length = 550)
    feature = feature.flatten()
    variance = (np.var(feature, axis=0) + 1e-8)
    feature -= (np.mean(feature, axis=0) + 1e-8)
    feature /= np.sqrt(variance)                  # feature normalization(important for SVM)
    feature_arr[:,i] = feature
feature_arr = feature_arr.T
print("Positive feature data shape",feature_arr.shape)

#for negative examples
feature_arr_neg = np.zeros((np.size(features),neg_data.shape[1]))
for i in range(neg_data.shape[1]):
    feature = librosa.feature.mfcc(y=neg_data[:,i],sr=44100,n_mfcc=12,n_fft=880,n_mels = 64,hop_length = 550)
    feature = feature.flatten()
    variance = (np.var(feature, axis=0) + 1e-8)
    feature -= (np.mean(feature, axis=0) + 1e-8)
    feature /= np.sqrt(variance)                  # feature normalization
    feature_arr_neg[:,i] = feature
feature_arr_neg = feature_arr_neg.T
print("Negative feature data shape",feature_arr_neg.shape)

Positive feature data shape (1018, 972)
Negative feature data shape (1018, 972)


In [19]:
# One class SVM(unsupervised) using only positive (just a toy example anyway)
permute = list(np.random.permutation(1018))
feature_arr = feature_arr[permute,:]            # random permutation
X_train = feature_arr[0:700,:]                  # train,test spilt
X_test = feature_arr[700:1018,:]
clf = skl.svm.OneClassSVM(nu=0.1, kernel="rbf",gamma = 'auto')     #nu sets the target training error
clf.fit(X_train)                                                   # fit the classifier
# make predictions on training and test data
y_pred_train = clf.predict(X_train)                                
y_pred_test = clf.predict(X_test)
y_pred_test_neg = clf.predict(feature_arr_neg)
# print out the error
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_test_neg = y_pred_test_neg[y_pred_test_neg == -1].size
print("Positive example training error:",n_error_train/700)
print("Positive example test error",n_error_test/(1018-700))
print("Negative example test error",1 - n_error_test_neg/1018)

Positive example training error: 0.09857142857142857
Positive example test error 0.1320754716981132
Negative example test error 0.0
